In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from pprint import pprint
from math import ceil
from time import time

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
print("TF version : ", tf.__version__)
print("Keras version : ", keras.__version__)
with_cuda = tf.test.is_built_with_cuda()
with_gpu = tf.test.is_gpu_available()
print("Built with CUDA : ", with_cuda)
print("Available GPU : ", with_gpu)

if with_cuda and with_gpu:
    keras.backend.set_image_data_format('channels_first')
else: 
    keras.backend.set_image_data_format('channels_last')

print("keras data_format : ", keras.backend.image_data_format())

TF version :  1.10.0
Keras version :  2.1.6-tf
Built with CUDA :  True
Available GPU :  True
keras data_format :  channels_first


In [4]:
BATCH_SIZE = 128
EPOCHS = 200
INIT_LR = 0.1

In [5]:
print("... loading CIFAR10 dataset ...")
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
y_train = np.squeeze(y_train)
y_test = np.squeeze(y_test)

x_train, y_train = shuffle(x_train, y_train, random_state=51)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,
                                                  test_size=0.2,
                                                  stratify=y_train,
                                                  random_state=51)
# cast samples and labels
x_train = x_train.astype(np.float32)
x_val = x_val.astype(np.float32)
x_test = x_test.astype(np.float32)
y_train = keras.utils.to_categorical(y_train.astype(np.int32), num_classes=10)
y_val = keras.utils.to_categorical(y_val.astype(np.int32), num_classes=10)
y_test = keras.utils.to_categorical(y_test.astype(np.int32), num_classes=10)

print("\tTRAIN - images {} | {}  - labels {} - {}".format(x_train.shape, x_train.dtype, y_train.shape, y_train.dtype))
print("\tVAL - images {} | {}  - labels {} - {}".format(x_val.shape, x_val.dtype, y_val.shape, y_val.dtype))
print("\tTEST - images {} | {}  - labels {} - {}\n".format(x_test.shape, x_test.dtype, y_test.shape, y_test.dtype))

... loading CIFAR10 dataset ...
	TRAIN - images (40000, 3, 32, 32) | float32  - labels (40000, 10) - float32
	VAL - images (10000, 3, 32, 32) | float32  - labels (10000, 10) - float32
	TEST - images (10000, 3, 32, 32) | float32  - labels (10000, 10) - float32



In [6]:
generator_aug = keras.preprocessing.image.ImageDataGenerator(samplewise_center=True,
                                                             samplewise_std_normalization=True,
                                                             width_shift_range=5,
                                                             height_shift_range=5,
                                                             fill_mode='constant',
                                                             cval=0.0,
                                                             horizontal_flip=True,
                                                             vertical_flip=False,
                                                             data_format=keras.backend.image_data_format())

generator = keras.preprocessing.image.ImageDataGenerator(samplewise_center=True,
                                                         samplewise_std_normalization=True,
                                                         data_format=keras.backend.image_data_format())

# python iterator object that yields augmented samples 
iterator_train_aug = generator_aug.flow(x_train, y_train, batch_size=BATCH_SIZE)

# python iterators object that yields not augmented samples 
iterator_train = generator.flow(x_train, y_train, batch_size=BATCH_SIZE)
iterator_valid = generator.flow(x_val, y_val, batch_size=BATCH_SIZE)
iterator_test = generator.flow(x_test, y_test, batch_size=BATCH_SIZE)

steps_per_epoch_train = int(ceil(iterator_train.n/BATCH_SIZE))
steps_per_epoch_val = int(ceil(iterator_valid.n/BATCH_SIZE))
steps_per_epoch_test = int(ceil(iterator_test.n/BATCH_SIZE))

In [7]:
# test iterator with data augmentation
x, y = iterator_train_aug.next()

print("x : {} | {}".format(x.shape, x.dtype))
print("y : {} | {}".format(y.shape, y.dtype))

x : (128, 3, 32, 32) | float32
y : (128, 10) | float32


# Build Keras model

In [8]:
from resnet import ResNet56

In [9]:
shape = [3, 32, 32] if keras.backend.image_data_format()=='channels_first' else [32, 32, 3]

model = ResNet56(input_shape=shape, classes=10, p=0.95, activation='drop-activation').build_model()

Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`


In [10]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3, 32, 32)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 16, 30, 30)   432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 16, 30, 30)   64          conv2d[0][0]                     
__________________________________________________________________________________________________
drop_activation_keras (DropActi (None, 16, 30, 30)   0           batch_normalization[0][0]        
__________________________________________________________________________________________________
conv2d_2 (

In [11]:
optimizer = keras.optimizers.SGD(lr=INIT_LR, momentum=0.9)
loss = 'categorical_crossentropy'
metrics = ['acc', ]
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [15]:
# model.save_weights('random_weights.h5')
model.load_weights('random_weights.h5')

# Train model

In [13]:
callbacks = []

model_ckpt = keras.callbacks.ModelCheckpoint("model_ckpt_dropactivation_best_drop-activation.h5",
                                             monitor='val_acc', verbose=1, save_best_only=True, 
                                             save_weights_only=True)
callbacks.append(model_ckpt)

def schedule(epoch):
    if epoch < 91:
        return INIT_LR
    if epoch < 136:
        return 0.1*INIT_LR
    if epoch < 182:
        return 0.01*INIT_LR
    else:
        return 0.001*INIT_LR
    
lr_schedule = keras.callbacks.LearningRateScheduler(schedule, verbose=1)
callbacks.append(lr_schedule)

In [16]:
%%time

history = model.fit_generator(iterator_train_aug, 
                              steps_per_epoch=steps_per_epoch_train,
                              epochs=EPOCHS,
                              verbose=1,
                              validation_data=iterator_valid,
                              validation_steps=steps_per_epoch_val,
                              callbacks=callbacks)

Epoch 1/200
313/313 [==============================] - 37s 117ms/step - loss: 2.3909 - acc: 0.4006 - val_loss: 2.9892 - val_acc: 0.3118
Epoch 2/200
313/313 [==============================] - 36s 115ms/step - loss: 1.8441 - acc: 0.5643 - val_loss: 2.5630 - val_acc: 0.4196
Epoch 3/200
313/313 [==============================] - 36s 115ms/step - loss: 1.5773 - acc: 0.6258 - val_loss: 2.4993 - val_acc: 0.4226
Epoch 4/200
313/313 [==============================] - 36s 114ms/step - loss: 1.3936 - acc: 0.6652 - val_loss: 2.6612 - val_acc: 0.4120
Epoch 5/200
313/313 [==============================] - 36s 114ms/step - loss: 1.2434 - acc: 0.7007 - val_loss: 1.8041 - val_acc: 0.5473
Epoch 6/200
313/313 [==============================] - 36s 114ms/step - loss: 1.1304 - acc: 0.7288 - val_loss: 1.3491 - val_acc: 0.6576
Epoch 7/200
313/313 [==============================] - 36s 114ms/step - loss: 1.0376 - acc: 0.7528 - val_loss: 1.4706 - val_acc: 0.6020
Epoch 8/200
313/313 [===========================

KeyboardInterrupt: 